In [1]:
## Loading Config Files

import os 
import json
import sys

sys.path.append(os.path.dirname(os.getcwd()))
with open('../config/runner_config.json') as json_file:
    json_data = json.load(json_file)


In [2]:
## Data Aggreagator Run
from deployment.Data_Retreiver import Data_Retreiver
from deployment.Data_Aggregator import Aggregator

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

sql_table = json_data["sql_aggregate"]
sql_table_raw = json_data["sql_raw_energy"]
sql_table_state =  json_data["sql_raw_energy"]

# Create Threads for All the devices
aggre = Aggregator(devices,sql_table,sql_table_raw,sql_table_state,sql_addr,sql_port, sql_user, sql_pw, sql_db)

#Do step and get the final output that would be printed to the log
aggre.do_step()
print(aggre.getLatest())

---Removals of Data:Raw_Energy:0 Raw_State0 Aggregate0

Aggregator: ---Removals of Data:Raw_Energy:0 Raw_State0 Aggregate0
2020-09-22 11:42:26.954213:    Starting Routine


In [3]:
## Conctrol Enactor 
from deployment.Control_Enactor import Enactor


devices = json_data["devices"]

sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

# Run Examples
print("Retreive - Set Points")
print(enact.retreive_previous_plan_DC(list(devices.keys())[1]))
print(enact.retreive_previous_plan_AC(list(devices.keys())[9]))
print("Enact Set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Same set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Diffrent set points")
print(enact.enact_light_plan(list(devices.keys())[1], 4320, 4320))
print(enact.enact_socket_plan(list(devices.keys())[9], 1200000))


Retreive - Set Points
(4320, 4320)
1200000
Enact Set points
True
True
Enact Same set points
False
False
Enact Diffrent set points
True
True


In [4]:
#Data retreiver Run
import datetime
import pandas as pd 
from deployment.Data_Retreiver import Data_Retreiver
devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

print("--Retreive filled data--\n",data.get_unsent_logs(datetime.datetime.now()))
print("--Retreive filled data--\n",data.retreive_filled_aggre(datetime.datetime.now() - datetime.timedelta(days=20), 2))

dev_light = "Nursery_1A_CPE_No_1"
dev_socket = "Playground_AC_socket_No_1"
group = ["Nursery_1A_CPE_No_1", "Nursery_1A_CPE_No_2", "Nursery_1B_CPE_No_3", "Nursery_1B_CPE_No_4",
         "Nursery_1C_CPE_No_5", "Nursery_1C_CPE_No_6"]
hour = 6

pd.set_option('display.max_columns', 30)
dt_ts = datetime.datetime.now()
print("--Forecast--\n", data.retreive_latest_forecast(dt_ts).head(5))
print("--Raw Sys--\n", data.retreive_latest_raw_system_snapshot(dt_ts).head(5))
print("--Priority--\n", data.retreive_latest_priority(dt_ts).head(5))
print("--Group Energy--\n", data.get_total_energy_for_group(group, dt_ts))
print("--Aggregated Values --\n", data.retreive_aggregared_values(dt_ts))
print("--AVG for Hour Values --\n", data.retreive_average_vals_for_hour(dt_ts, hour,8))
print("--Average P Light--\n", data.retreive_average_P_lights(dev_light, dt_ts))
print("--AC Session--\n", data.retreive_AC_Session(dev_socket, dt_ts))
print("--AC Energy--\n", data.retreive_AC_Energy(dev_socket, dt_ts))
print("--Light Session--\n", data.retreive_Light_Session(dev_light, dt_ts))
print("--Light Energy--\n", data.retreive_Light_Energy(dev_light, dt_ts))
print("--Quota Latest Light--\n", data.get_latest_quota(dev_light, dt_ts))
print("--Quota Latest Socket --\n", data.get_latest_quota(dev_socket, dt_ts))

--Retreive filled data--
    id                        timestamp  transmitted  message_id   mem   cpu  \
0  11 2020-09-22 11:38:55.356447+00:00        False         1.0  51.8  45.3   

   storage_perc  storage_free  temperature  user_actions  ...  \
0          41.3     259.92432          0.0           1.0  ...   

   quota1_streetlight_no_2  quota2_streetlight_no_2  quota1_streetlight_no_3  \
0                   4329.0                   4320.0                   4329.0   

   quota2_streetlight_no_3  priority_nursery1_lights  \
0                   4320.0                       1.0   

   priority_nursery1_sockets  priority_nursery2_lights  \
0                        2.0                       3.0   

   priority_nursery2_sockets  priority_playground_lights  \
0                        4.0                         5.0   

   priority_playground_sockets  
0                          6.0  

[1 rows x 142 columns]
Value Error probably not data avilable for period:>  2020-09-02 11:30:00   2  Erro

In [5]:
from deployment.Battery import Battery

bat = Battery(state_of_charge=94.0, battery_capacity=18.79, max_discharge=40.0, charge_efficiency=0.85)
print(bat)
print("---Storage Capacity---\n", bat.get_storage_capacity_left())
print("---Discharge Capacity---\n", bat.get_discharge_capacity_left())
print("---Discharge Battery---\n", bat.discharge_battery(1.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(0.4))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)


SoC: 94.0 Energy_Left: 10.1466
---Storage Capacity---
 1.3263529411764705
---Discharge Capacity---
 10.1466
---Discharge Battery---
 0.0
SoC: 83.88823842469398 Energy_Left: 8.246599999999999
---Charge Battery---
 0.0
SoC: 85.6977115486961 Energy_Left: 8.586599999999999
---Discharge Battery---
 0.0
SoC: 70.26397019691323 Energy_Left: 5.686599999999997
---Discharge Battery---
 0.0
SoC: 54.83022884513037 Energy_Left: 2.786599999999997
---Discharge Battery---
 0.11340000000000305
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Charge Battery---
 0.0
SoC: 59.90420436402342 Energy_Left: 3.7400000000000007
---Charge Battery---
 0.0
SoC: 79.80840872804684 Energy_Left: 7.480000000000001
---Charge Battery---
 0.0
SoC: 99.71261309207026 Energy_Left: 11.22
---Charge Battery---
 4.336470588235296
SoC: 100.0 Energy_Left: 11.274
---Charge Battery---
 4.4
SoC: 100.0 Energy_Left: 11.274


In [6]:
# Controller Run

#Data retreiver Run
import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Control_Enactor import Enactor
from deployment.Controller import Controller

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

## Conctrol Enactor 
sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

#Controller Init
ts_step = datetime.datetime.now()  # - datetime.timedelta(hours=3)

allocation = json_data["allocation"]
if 'System_Data' in allocation:
    del allocation['System_Data']

control = Controller(data, enact, allocation, 4)
print(control.getLatest())

control.do_step(ts_step)
print(control.getLatest())

Controller : Initilised
Controller : Latest Data From: 2020-09-22 11:42:36.353445
-------Energy State--------
Generated energy: 1877.3041473989454
System Load: 2941.2929117269196
Battery SoC: 100.0
Remaining Energy: 9741.752653326643
{1: 'nursery1_lights', 2: 'nursery1_sockets', 3: 'nursery2_lights', 4: 'nursery2_sockets', 5: 'playground_lights', 6: 'playground_sockets'}
------------------------------------------------
For Device: nursery1_lights with energy avialable: 9741.752653326643
Decisions: {'state': 'Unconstrained', 'energy_est_used_total': 307.255056327603, 'constraining_factor': 1.0, 'device_const': {'Nursery_1A_CPE_No_1': (4320, 4320, False), 'Nursery_1A_CPE_No_2': (4320, 4320, False), 'Nursery_1B_CPE_No_3': (4320, 4320, False), 'Nursery_1B_CPE_No_4': (4320, 4320, False), 'Nursery_1C_CPE_No_5': (4320, 4320, False), 'Nursery_1C_CPE_No_6': (4320, 4320, False)}, 'timestamp': '2020-09-22 11:42:36.353445'}
Remaining: 9373.046585733518
---------------------------------------------

In [7]:
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster import Forecaster

devices = json_data["devices"]
devices = {x: devices[x] for x in devices if x != "Victron_VenusGX"}
# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.latest_forecast)

Running Forecasting --  2020-09-22 11:42:56.924593  --
Forecaster : 2020-09-22 11:43:02.564734 : [{"system_load":80.9638598053,"generated_energy":423.3098144531,"consumed_energy":99.5815429688}]


parameter,nursery_1a_cpe_no_1,nursery_1a_cpe_no_2,nursery_1b_cpe_no_3,nursery_1b_cpe_no_4,nursery_1c_cpe_no_5,nursery_1c_cpe_no_6,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_10,nursery_2b_cpe_no_9,nursery_2c_cpe_no_11,nursery_2c_cpe_no_12,nursery_ac_socket_1a_no_1,nursery_ac_socket_1a_no_2,nursery_ac_socket_1b,...,playground_ac_socket_no_1,playground_no_1,playground_no_2,playground_no_3,playground_no_4,playground_no_5,streetlight_no_1,streetlight_no_2,streetlight_no_3,battery_soc,charged_energy,consumed_energy,generated_energy,timestamp,system_load
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-22 11:00:00,0.030700,0.000000,0.000000,0.000000,0.016257,0.000000,0.016738,0.000000,0.018124,0.035014,0.0,0.000000,0.967857,0.241379,1.878571,...,1.197972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,614.202393,71.543457,685.745850,2020-09-22 11:00:00,56.350090
2020-09-22 12:00:00,0.000000,0.000000,0.000000,0.000000,0.016645,0.000000,0.016198,0.000000,0.114793,0.027637,0.0,0.000000,1.166872,0.247166,1.757266,...,1.347253,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,323.728271,99.581543,423.309814,2020-09-22 12:00:00,80.963860
2020-09-22 13:00:00,0.000000,0.000000,0.000000,0.000000,0.014613,0.000000,0.016983,0.000981,0.068952,0.016466,0.0,0.000000,0.973484,0.185587,1.987685,...,1.253212,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,220.756241,67.295212,288.051453,2020-09-22 13:00:00,50.880878
2020-09-22 14:00:00,0.000000,0.000000,0.000000,0.015976,0.016403,0.000000,0.014026,0.000000,0.001530,0.006013,0.0,0.000000,0.452176,0.096726,1.941742,...,1.725673,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,121.215000,112.100325,233.315325,2020-09-22 14:00:00,47.525956
2020-09-22 15:00:00,0.000000,0.000000,0.000000,0.000000,0.014172,0.000000,0.016975,0.000000,0.001980,0.022305,0.0,0.000000,0.205782,0.083333,1.247449,...,1.216601,7.065510,6.282747,6.909766,6.699572,6.039888,7.008700,4.537200,5.790584,100.000000,117.214615,66.087265,183.301880,2020-09-22 15:00:00,0.831776
2020-09-22 16:00:00,2.671159,0.000000,2.733462,0.000000,2.902771,0.000000,3.124074,0.000000,7.793032,8.470631,0.0,2.795987,0.140394,0.000000,0.685345,...,1.411111,12.523463,12.719450,12.692782,12.369589,12.464101,12.647728,12.892938,12.856978,100.000000,75.739208,62.905918,138.645126,2020-09-22 16:00:00,-77.983259
2020-09-22 17:00:00,4.777869,0.000000,4.862384,0.000000,4.835743,0.033594,4.798266,0.000849,13.844206,14.349727,0.0,4.783312,0.184259,0.000000,0.208333,...,0.600980,12.167298,12.812594,12.743391,12.228749,12.254019,12.443326,12.506397,12.803812,99.566830,-91.398907,138.099106,46.700199,2020-09-22 17:00:00,-20.916526
2020-09-22 18:00:00,4.909534,0.000000,5.040962,0.000000,4.674912,0.000000,4.988470,0.000000,14.245329,14.422339,0.0,4.595859,0.195833,0.000000,0.174405,...,0.390625,12.108186,12.739422,12.801451,11.943931,12.468716,12.253148,12.469222,12.612388,98.262234,-275.269651,275.703125,0.433474,2020-09-22 18:00:00,116.648246
2020-09-22 19:00:00,4.615828,0.000000,4.818479,0.000000,4.657596,0.000000,4.819996,0.000000,14.419591,14.553560,0.0,4.537176,0.237548,0.000000,0.179803,...,0.519298,11.979742,12.607411,12.567817,12.072528,12.196911,12.439433,12.072023,12.617783,96.762045,-316.539917,316.539917,0.000000,2020-09-22 19:00:00,159.210742


In [8]:
##Forecaster V2 - ML and PV Models
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster_v2 import Forecaster

devices = json_data["devices"]
devices = {x: devices[x] for x in devices if x != "Victron_VenusGX"}
# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.full_df)

Running Forecasting --  2020-09-22 11:43:03.473314  --
Forecaster : 2020-09-22 11:44:52.362750 : [{"system_load":56.6073295439,"generated_energy":1584.2405333184,"consumed_energy":75.1499280861}]


,consumed_energy,timestamp,system_load,streetlight_no_1,streetlight_no_2,streetlight_no_3,playground_no_1,playground_no_2,playground_no_3,playground_no_4,playground_no_5,playground_ac_socket_no_1,playground_ac_socket_no_2,nursery_ac_socket_1a_no_1,nursery_ac_socket_1a_no_2,...,nursery_1c_cpe_no_5,nursery_1c_cpe_no_6,nursery_ac_socket_2a_no_1,nursery_ac_socket_2a_no_2,nursery_ac_socket_2b,nursery_ac_socket_2c,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_9,nursery_2b_cpe_no_10,nursery_2c_cpe_no_11,nursery_2c_cpe_no_12,generated_energy,charged_energy,battery_soc
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-22 12:00:00,64.801535,2020-09-22 12:00:00,50.047984,1.692408e-31,-9.352258e-38,2.944371e-35,-3.159344e-32,-1.994862e-36,2.627728e-30,1.398875e-36,-3.293399e-29,1.347516,3.075004,1.167118,0.280114,...,0.006000,1.248519e-43,2.829416,3.601442,3.251486,1.155894,5.000005e-03,0.0,-0.303161,1.249008,0.0,8.573466e-28,1303.092333,1238.290797,4.98838
2020-09-22 13:00:00,59.076798,2020-09-22 13:00:00,49.889875,-1.360945e-23,3.193585e-32,-4.426347e-31,-5.514668e-32,-3.228256e-31,5.169726e-24,9.734880e-37,4.614041e-24,1.410675,1.823098,0.844905,0.183754,...,0.005000,1.543725e-37,2.797756,3.744431,3.311588,1.067864,5.999999e-03,0.0,-0.608966,0.655828,0.0,9.322363e-28,932.041083,872.964284,8.50506
2020-09-22 14:00:00,54.846682,2020-09-22 14:00:00,51.614762,-2.584006e-14,2.870951e-16,2.345150e-16,4.205470e-15,3.521235e-20,7.008317e-15,1.666514e-37,5.938703e-15,1.727915,3.367200,0.451647,0.110452,...,0.005000,1.908731e-31,2.286684,2.863326,2.118161,0.234393,5.999999e-03,0.0,-0.963406,-0.210498,0.0,2.267197e-28,504.247566,449.400884,10.3154
2020-09-22 15:00:00,51.147308,2020-09-22 15:00:00,33.725430,2.335953e+00,1.511999e+00,1.929964e+00,2.355002e+00,2.094004e+00,2.303008e+00,2.232993e+00,2.013003e+00,1.216660,6.302930,0.205483,0.075962,...,0.005000,2.360455e-25,1.974123,2.415437,1.337908,0.001033,5.999999e-03,0.0,-0.794273,-0.216768,0.0,5.492763e-15,93.240107,42.092799,10.485
2020-09-22 16:00:00,42.914968,2020-09-22 16:00:00,8.390675,4.215915e+00,4.297996e+00,4.285920e+00,4.174003e+00,4.240009e+00,4.231014e+00,4.122986e+00,4.155006e+00,1.411394,2.883163,0.139434,-0.011049,...,0.967996,1.024459e-18,1.112672,1.806935,1.234942,0.000996,1.040994e+00,0.0,160.470029,101.260189,0.0,9.320039e-01,0.000000,-42.914968,40
2020-09-22 17:00:00,191.906407,2020-09-22 17:00:00,81.146357,4.147916e+00,4.168997e+00,4.267921e+00,4.056003e+00,4.271009e+00,4.248014e+00,4.075986e+00,4.085006e+00,0.597873,2.775858,0.183455,-0.010949,...,1.611993,1.100003e-02,0.860654,1.242044,0.951900,0.000958,1.598992e+00,0.0,272.589926,180.072808,0.0,1.594007e+00,0.000000,-191.906407,40
2020-09-22 18:00:00,466.134728,2020-09-22 18:00:00,222.560099,4.083917e+00,4.155997e+00,4.203922e+00,4.036003e+00,4.246009e+00,4.267014e+00,3.980987e+00,4.156006e+00,0.386904,3.452381,0.195456,-0.010871,...,1.557994,1.702462e-18,0.630471,1.096491,0.838930,0.037502,1.662991e+00,0.0,273.868059,185.264693,0.0,1.532006e+00,0.000000,-466.134728,40
2020-09-22 19:00:00,547.216203,2020-09-22 19:00:00,265.416968,4.145916e+00,4.023997e+00,4.205922e+00,3.993003e+00,4.202009e+00,4.189014e+00,4.023986e+00,4.066006e+00,0.515375,2.801148,0.237477,-0.010812,...,1.552994,1.181573e-28,0.679196,1.035195,0.742968,0.214433,1.606991e+00,0.0,276.297761,187.565141,0.0,1.512006e+00,0.000000,-547.216203,40
2020-09-22 20:00:00,546.814943,2020-09-22 20:00:00,264.347335,4.142916e+00,4.073997e+00,4.207922e+00,3.927003e+00,4.165009e+00,4.132014e+00,4.022987e+00,4.033006e+00,0.607698,3.888942,0.102385,-0.010770,...,1.608993,-9.448226e-36,0.468860,0.857901,0.555967,0.759143,1.602991e+00,0.0,275.754011,184.981204,0.0,1.628007e+00,0.000000,-546.814943,40


In [9]:
##Remote Logger Example

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Remote_Logger import RemoteLogger

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

mqtt_address = json_data["mqtt_address"]
mqtt_port = json_data["mqtt_port"]
mqtt_username = json_data["mqtt_username"]
mqtt_password = json_data["mqtt_password"]

data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

rl = RemoteLogger(data, mqtt_address, mqtt_port, mqtt_username, mqtt_password)

rl.do_step()
print(rl.getLatest())

Remote Logger: 2020-09-22 11:44:52.614526 : Publishing message with Nulls:['quota2_nursery_ac_socket_1a_no_1', 'quota2_nursery_ac_socket_1a_no_2', 'quota2_nursery_ac_socket_1b', 'quota2_nursery_ac_socket_1c', 'quota2_nursery_ac_socket_2a_no_1', 'quota2_nursery_ac_socket_2a_no_2', 'quota2_nursery_ac_socket_2b', 'quota2_nursery_ac_socket_2c', 'quota2_playground_ac_socket_no_1', 'quota2_playground_ac_socket_no_2']  with return:(0, 1)
